In [1]:
import bambi as bmb
import numpy as np
import pandas as pd

In [2]:
#%load_ext autoreload
#%autoreload 2

In [3]:
data = pd.read_csv("bambi/tests/data/inhaler.csv")
data["rating"] = pd.Categorical(data["rating"], categories=[1, 2, 3, 4])
data

,subject,rating,treat,period,carry
0,1,1,0.5,0.5,0
1,2,1,0.5,0.5,0
2,3,1,0.5,0.5,0
3,4,1,0.5,0.5,0
4,5,1,0.5,0.5,0
...,...,...,...,...,...
567,282,1,-0.5,-0.5,-1
568,283,1,0.5,-0.5,1
569,284,1,0.5,-0.5,1
570,285,2,-0.5,-0.5,-1


In [4]:
df = data.groupby(["treat", "carry", "rating"], as_index=False).size()
df = df.pivot(index=["treat", "carry"], columns="rating", values="size").reset_index()
df.columns = ["treat", "carry", "y1", "y2", "y3", "y4"]
df

,treat,carry,y1,y2,y3,y4
0,-0.5,-1,71,63,5,3
1,-0.5,0,76,55,10,3
2,-0.5,1,0,0,0,0
3,0.5,-1,0,0,0,0
4,0.5,0,99,41,0,2
5,0.5,1,112,30,2,0


In [5]:
# Intercept only
model = bmb.Model("c(y1, y2, y3, y4) ~ 1", df, family="multinomial")
idata = model.fit()

/home/tomas/anaconda3/envs/bambi/lib/python3.10/site-packages/aesara/tensor/nnet/basic.py:1116: FutureWarning: Softmax now accepts an axis argument. For backwards-compatibility it defaults to -1 when not specified, but in the future the default will be `None`.
To suppress this warning specify axis explicitly.
  warnings.warn(
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 2 jobs)
NUTS: [Intercept]


Sampling 2 chains for 1_000 tune and 1_000 draw iterations (2_000 + 2_000 draws total) took 2 seconds.


In [6]:
idata

Inference data with groups:
	> posterior
	> log_likelihood
	> sample_stats
	> observed_data

In [8]:
model.predict(idata, inplace=False)

Inference data with groups:
	> posterior
	> log_likelihood
	> sample_stats
	> observed_data

In [13]:
idata_2 = model.predict(idata, kind="pps", inplace=False)

[142 144   0   0 142 144]


In [14]:
idata_2

Inference data with groups:
	> posterior
	> posterior_predictive
	> log_likelihood
	> sample_stats
	> observed_data

In [12]:
data = pd.DataFrame(
    {
        "state": ["A", "B", "C"],
        "y1": [35298, 1885, 5775],
        "y2": [167328, 20731, 21564],
        "y3": [212682, 37716, 20222],
        "y4": [37966, 5196, 3277],
    }
)

# Contains group-specific effect
model = bmb.Model(
    "c(y1, y2, y3, y4) ~ 1 + (1 | state)", data, family="multinomial", noncentered=False
)
idata = model.fit(tune=1000, draws=1000, random_seed=0)

/home/tomas/anaconda3/envs/bambi/lib/python3.10/site-packages/aesara/tensor/nnet/basic.py:1116: FutureWarning: Softmax now accepts an axis argument. For backwards-compatibility it defaults to -1 when not specified, but in the future the default will be `None`.
To suppress this warning specify axis explicitly.
  warnings.warn(
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 2 jobs)
NUTS: [Intercept, 1|state_sigma, 1|state]


Sampling 2 chains for 1_000 tune and 1_000 draw iterations (2_000 + 2_000 draws total) took 252 seconds.
The acceptance probability does not match the target. It is 0.886, but should be close to 0.8. Try to increase the number of tuning steps.
The chain reached the maximum tree depth. Increase max_treedepth, increase target_accept or reparameterize.
The chain reached the maximum tree depth. Increase max_treedepth, increase target_accept or reparameterize.


In [13]:
from bambi.families.multivariate import Multinomial, MultivariateFamily
isinstance(model.family, MultivariateFamily)

True

In [15]:
model.predict(idata)

In [16]:
model.predict(idata, kind="pps")

In [17]:
idata

Inference data with groups:
	> posterior
	> posterior_predictive
	> log_likelihood
	> sample_stats
	> observed_data